In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [3]:
loans = pd.read_csv(r"C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 3\lending-club-data.csv")

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
loans.head()[:2]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.1435,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.3932,20161201T000000,1,1,1


In [5]:
list(loans)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec

In [6]:
loans["safe_loans"] = [-1 if x==1 else 1 for x in loans["bad_loans"]]

In [7]:
loans.drop("bad_loans",axis=1,inplace=True)

In [8]:
def get_count(values):
    return len(values)
def as_perc(value, total):
    return value/float(total)
x = loans.groupby("safe_loans").size()
y = x.apply(lambda x: x*100./len(loans))

In [9]:
y

safe_loans
-1    18.881467
 1    81.118533
dtype: float64

In [10]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]

target = 'safe_loans'                    # prediction target (y) (+1 means safe,

In [11]:
loans = loans[features+[target]]

In [12]:
train_idx = pd.read_json(r"C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 3\module-5-assignment-1-train-idx.json")
validation_idx = pd.read_json(r"C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 3\module-5-assignment-1-validation-idx.json")

In [13]:
loans.head()[:2]

,grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loans
0,B,B2,0,11,RENT,27.65,credit_card,36 months,1,1,83.7,0.0,1
1,C,C4,1,1,RENT,1.00,car,60 months,1,1,9.4,0.0,-1


In [14]:
catcols = []
for x in list(loans):
    if ((loans[x].dtype != "int64") & (loans[x].dtype != "float64")):
        catcols.append(x)
catcols

['grade', 'sub_grade', 'home_ownership', 'purpose', 'term']

In [15]:
newnames = []
le = LabelEncoder()
for column in catcols:
    le.fit(loans[column])
    loans[column + "_cat"] = le.transform(loans[column])
    newnames.append(column+"_cat")

In [16]:
loans_dropped = loans[catcols]
loans.drop(catcols,axis=1,inplace=True)

In [17]:
colnames = []
for name in newnames:
    for i in loans[name].unique():
        colnames.append(name+"_"+str(i))
colnames       

['grade_cat_1',
 'grade_cat_2',
 'grade_cat_0',
 'grade_cat_4',
 'grade_cat_5',
 'grade_cat_3',
 'grade_cat_6',
 'sub_grade_cat_6',
 'sub_grade_cat_13',
 'sub_grade_cat_14',
 'sub_grade_cat_10',
 'sub_grade_cat_3',
 'sub_grade_cat_20',
 'sub_grade_cat_26',
 'sub_grade_cat_9',
 'sub_grade_cat_12',
 'sub_grade_cat_5',
 'sub_grade_cat_15',
 'sub_grade_cat_0',
 'sub_grade_cat_7',
 'sub_grade_cat_8',
 'sub_grade_cat_11',
 'sub_grade_cat_16',
 'sub_grade_cat_2',
 'sub_grade_cat_4',
 'sub_grade_cat_19',
 'sub_grade_cat_1',
 'sub_grade_cat_23',
 'sub_grade_cat_17',
 'sub_grade_cat_18',
 'sub_grade_cat_27',
 'sub_grade_cat_22',
 'sub_grade_cat_25',
 'sub_grade_cat_24',
 'sub_grade_cat_33',
 'sub_grade_cat_21',
 'sub_grade_cat_31',
 'sub_grade_cat_29',
 'sub_grade_cat_28',
 'sub_grade_cat_34',
 'sub_grade_cat_30',
 'sub_grade_cat_32',
 'home_ownership_cat_3',
 'home_ownership_cat_2',
 'home_ownership_cat_0',
 'home_ownership_cat_1',
 'purpose_cat_1',
 'purpose_cat_0',
 'purpose_cat_9',
 'purpose

In [18]:
enc = OneHotEncoder()
new_loans = pd.DataFrame(enc.fit_transform(loans[newnames]).toarray(), columns = colnames,dtype=int)

In [19]:
# loans.ix[:, loans.columns != 'b']
loans = pd.concat([loans[loans.columns.difference(newnames)],new_loans],axis=1)

In [20]:
train_data = loans.iloc[train_idx]
validation_data = loans.iloc[validation_idx]

In [21]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print( "Number of safe loans  : %s" % len(safe_loans_raw))
print ("Number of risky loans : %s" % len(risky_loans_raw))

Number of safe loans  : 99457
Number of risky loans : 23150


In [22]:
random.seed( 1 )

In [23]:
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
percentage

0.2327639080205516

In [24]:
risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(frac = percentage)

In [25]:
loans_data = pd.concat([risky_loans,safe_loans],axis=0)

In [26]:
print( "Number of safe loans  : %s" % len(safe_loans))
print ("Number of risky loans : %s" % len(risky_loans))
len(loans_data)

Number of safe loans  : 23150
Number of risky loans : 23150


46300

In [40]:
train_data, validation_data = train_test_split(loans_data, test_size = 0.2)

In [41]:
train_result = train_data[target]
train_data.drop(target,axis=1,inplace=True)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
validation_data.drop(target,axis=1,inplace=True)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [31]:
X = np.array(train_data)
Y = np.array(train_result)
decision_tree_model = DecisionTreeClassifier(max_depth=6)
small_model = DecisionTreeClassifier(max_depth=2)
decision_tree_model.fit(X,Y)
small_model.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [36]:
tree.export_graphviz(decision_tree_model,out_file='tree.dot')

In [44]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = pd.concat([sample_validation_data_safe,sample_validation_data_risky],axis=0)
sample_validation_data.drop(target,axis=1,inplace=True)

In [45]:
decision_tree_model.predict(sample_validation_data)

array([ 1, -1,  1, -1], dtype=int64)

In [46]:
decision_tree_model.predict_proba(sample_validation_data)

array([[ 0.17822266,  0.82177734],
       [ 0.55752577,  0.44247423],
       [ 0.45330296,  0.54669704],
       [ 0.55752577,  0.44247423]])

In [47]:
small_model.predict_proba(sample_validation_data)

array([[ 0.2312    ,  0.7688    ],
       [ 0.59440463,  0.40559537],
       [ 0.42542038,  0.57457962],
       [ 0.59440463,  0.40559537]])

In [49]:
decision_tree_model.score(X,Y)

0.63987580993520521

In [50]:
small_model.score(X,Y)

0.61325593952483803